In [23]:
import copy
import glob
import pickle
import warnings
from datetime import datetime, timedelta
from itertools import product
import joblib

import cartopy
import cartopy.crs as ccrs
import cartopy.feature
import cartopy.feature as cfeature
import cartopy.feature as cf
import cartopy.io.shapereader as shpreader
import matplotlib as mpl
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely.geometry as sgeom
import xarray as xr
from scipy import stats
from scipy.spatial.distance import cdist
from shapely import geometry
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances

# import sys
# sys.path.append("/glade/u/home/jhayron/WeatherRegimes/Scripts/")
# import cluster_analysis, narm_analysis, som_analysis

# Read data

In [2]:
ds_era_anomalies = xr.open_dataset('/glade/work/jhayron/Weather_Regimes/ERA5/Daily_1degree/anomalies/z500_anomalies_v1959_2010_detrended.nc')


In [3]:
df_WR = pd.read_csv('/glade/work/jhayron/Weather_Regimes/WR_Series_v3.csv',index_col=0,\
            parse_dates=True)

In [4]:
def get_cold_indx(ds, mo_init=9, mo_end=2):
    """
    Extract indices for cold season.
    Grabbing Sept thru February init, for Oct thru March predictions.
    """
    dt_array = pd.to_datetime(ds['time'])
    return xr.where((dt_array.month>=mo_init) | (dt_array.month<=mo_end), True, False)

# Get mean Geopotential, weeks 1, 2, 3, 4, 5

In [5]:
weekly_average = ds_era_anomalies.rolling(time=7,min_periods=4).mean().shift(time=-6)

## Only Mondays and Thursdays

In [6]:
# Friday, December 31, 2021: 0 --> monday, 6 --> sunday

In [7]:
dow = pd.to_datetime(weekly_average.time).day_of_week

In [8]:
where_mon_thu = np.where((dow == 0)|(dow==3))[0][:-18]

In [9]:
week1_anoms = weekly_average.isel(time=where_mon_thu)
week2_anoms = weekly_average.isel(time=where_mon_thu+7)
week3_anoms = weekly_average.isel(time=where_mon_thu+7*2)
week4_anoms = weekly_average.isel(time=where_mon_thu+7*3)
week5_anoms = weekly_average.isel(time=where_mon_thu+7*4)
week6_anoms = weekly_average.isel(time=where_mon_thu+7*5)
week7_anoms = weekly_average.isel(time=where_mon_thu+7*6)
week8_anoms = weekly_average.isel(time=where_mon_thu+7*7)
week9_anoms = weekly_average.isel(time=where_mon_thu+7*8)

In [10]:
week2_anoms['time'] = week1_anoms.time
week3_anoms['time'] = week1_anoms.time
week4_anoms['time'] = week1_anoms.time
week5_anoms['time'] = week1_anoms.time
week6_anoms['time'] = week1_anoms.time
week7_anoms['time'] = week1_anoms.time
week8_anoms['time'] = week1_anoms.time
week6_anoms['time'] = week1_anoms.time
week7_anoms['time'] = week1_anoms.time
week8_anoms['time'] = week1_anoms.time
week9_anoms['time'] = week1_anoms.time


# Get most frequent WR of each week

In [11]:
from scipy.stats import mode

In [12]:
df_WR = df_WR.resample('D').mean()

In [13]:
df_WR_week_mode = df_WR.rolling(7,min_periods=4).apply(lambda x: mode(x,keepdims=True)[0]).shift(-6)

In [14]:
week1_wr = df_WR_week_mode.iloc[where_mon_thu]
week2_wr = df_WR_week_mode.iloc[where_mon_thu+7]
week3_wr = df_WR_week_mode.iloc[where_mon_thu+7*2]
week4_wr = df_WR_week_mode.iloc[where_mon_thu+7*3]
week5_wr = df_WR_week_mode.iloc[where_mon_thu+7*4]
week6_wr = df_WR_week_mode.iloc[where_mon_thu+7*5]
week7_wr = df_WR_week_mode.iloc[where_mon_thu+7*6]
week8_wr = df_WR_week_mode.iloc[where_mon_thu+7*7]
week9_wr = df_WR_week_mode.iloc[where_mon_thu+7*8]

In [15]:
week2_wr.index = week1_wr.index
week3_wr.index = week1_wr.index
week4_wr.index = week1_wr.index
week5_wr.index = week1_wr.index
week6_wr.index = week1_wr.index
week7_wr.index = week1_wr.index
week8_wr.index = week1_wr.index
week9_wr.index = week1_wr.index

# Only cold months

In [16]:
cold_indx = get_cold_indx(week1_anoms,10,3)

In [17]:
week1_anoms = week1_anoms.isel(time=cold_indx)
week2_anoms = week2_anoms.isel(time=cold_indx)
week3_anoms = week3_anoms.isel(time=cold_indx)
week4_anoms = week4_anoms.isel(time=cold_indx)
week5_anoms = week5_anoms.isel(time=cold_indx)
week6_anoms = week6_anoms.isel(time=cold_indx)
week7_anoms = week7_anoms.isel(time=cold_indx)
week8_anoms = week8_anoms.isel(time=cold_indx)
week9_anoms = week9_anoms.isel(time=cold_indx)

In [18]:
week1_wr = week1_wr.iloc[cold_indx]
week2_wr = week2_wr.iloc[cold_indx]
week3_wr = week3_wr.iloc[cold_indx]
week4_wr = week4_wr.iloc[cold_indx]
week5_wr = week5_wr.iloc[cold_indx]
week6_wr = week6_wr.iloc[cold_indx]
week7_wr = week7_wr.iloc[cold_indx]
week8_wr = week8_wr.iloc[cold_indx]
week9_wr = week9_wr.iloc[cold_indx]

# Save files

In [19]:
week1_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week1_z500_anoms_v3.nc')
week2_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week2_z500_anoms_v3.nc')
week3_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week3_z500_anoms_v3.nc')
week4_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week4_z500_anoms_v3.nc')
week5_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week5_z500_anoms_v3.nc')
week6_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week6_z500_anoms_v3.nc')
week7_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week7_z500_anoms_v3.nc')
week8_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week8_z500_anoms_v3.nc')
week9_anoms.to_netcdf('/glade/work/jhayron/Weather_Regimes/weekly_anomalies/week9_z500_anoms_v3.nc')

In [20]:
week1_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week1_wr_v3.csv')
week2_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week2_wr_v3.csv')
week3_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week3_wr_v3.csv')
week4_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week4_wr_v3.csv')
week5_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week5_wr_v3.csv')
week6_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week6_wr_v3.csv')
week7_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week7_wr_v3.csv')
week8_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week8_wr_v3.csv')
week9_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/week9_wr_v3.csv')

# Compute weather patterns with the mean of the week

In [39]:
df_wr = pd.DataFrame(index = pd.to_datetime(week1_anoms.time))
distance_wr = pd.DataFrame(index = pd.to_datetime(week1_anoms.time))
for week, ds in zip(['week1','week2','week3','week4','week5','week6','week7','week8','week9'],\
    [week1_anoms,week2_anoms,week3_anoms,week4_anoms,week5_anoms,week6_anoms,week7_anoms,week8_anoms,week9_anoms]):
    # region for clustering
    lat0=10; lat1=70; lon0=210; lon1=320
    ds_era5_train = ds.where((ds.lat>=lat0)&(ds.lat<=lat1)&\
                       (ds.lon>=lon0)&(ds.lon<=lon1),drop=True)
    ds_era5_train = ds_era5_train.stack(flat=('lat','lon')).transpose('time','flat').z500_anomalies
    data_era5_train = ds_era5_train.data

    import joblib
    pca_obj = joblib.load(r'/glade/work/jhayron/Weather_Regimes/models/PCA_ERA5_v4.mdl')
    k_means = joblib.load(r'/glade/work/jhayron/Weather_Regimes/models/KMeans_ERA5_v4.pkl')
    data_era5_train = pca_obj.transform(data_era5_train)
    
    euc_res=euclidean_distances(k_means.cluster_centers_, data_era5_train)
    df_wr[week] = k_means.predict(data_era5_train)
    distance_wr[week] = euc_res.min(axis=0)

In [22]:
df_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/weekly_wr_mean_geop_v3.csv')

In [40]:
distance_wr.to_csv('/glade/work/jhayron/Weather_Regimes/weekly_wr/weekly_distance_mean_geop_v3.csv')

array([3.44966534, 4.48207571, 4.74829823, ..., 3.65037026, 2.77871185,
       2.47167445])

In [35]:
k_means.predict(data_era5_train)

array([1, 1, 1, ..., 3, 1, 2], dtype=int32)